## Project 2 - Data Wrangling

### Project Objectives

- Using `requests` and `BeautifulSoup` to scrape a simple web page.
- Wrangle unstructured text data into a DataFrame format.
- Use `Pandas` functions to extract information from a DataFrame.
- Introduced to the `Folium` package.

## Setup Notebook

In [60]:
# Import 3rd party libraries such as:
# os- which is the operating system dependent function
# requests- a function that sends HTTP requests through python
# pandas- a library function for manipulating and analysing data
# beautifulsoup- a library package for parsing HTML and XML information
# and matplotlib.pylab- a function for creating statistical plots and visualizations

import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pylab as plt

# Import local libraries
import utils

# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

Install a super cool geospatial plotting package `Folium`.

In [61]:
# using the pip package installer method, install python library
!pip install folium

In [62]:
# imported python library can be called folium- a library function that visualizes data in interactive map
import folium

# Overview
You start a new job at an engineering company and you join a team that is planning to build a new waterfront facility on Lake Ontario. A critical input to the team's design process is information about the swell height and its seasonality (We'll learn more about seasonality in Week 5). Your manager informs you that NOAA (National Oceanic and Atmospheric Administration) has an array of sensors installed throughout the great lakes that measure swell height and direction among other things. 

In this project, you'll be working with the [NOAA - Great Lakes Environmental Research Laboratory (GLERL)](https://www.glerl.noaa.gov/res/glcfs/) dataset which contains forecasts and measurements for Ice Cover, Wave Height, Current Direction, Wind Speed, and others. We have already worked with this dataset in Lecture 3.2. 

<br>
<img src="images/noaa.gif" alt="drawing" width="500"/>
<br>

Your managers asks you to programmatically pull all available wave data from the GLERL server and report to her the minimum, maximum, and average wave height from the three closest grid points in the dataset to the planned location of the facility (lat = 43.9593°, lon = -78.1677°).  

The image below shows the grid point locations in black and the location of the planned facility in red.

In [68]:
# Import 'grid_plot.csv'
grid_plot = pd.read_csv('grid_plot.csv')

# Create a map of Toronto
map1 = folium.Map(location=[43.9593, -78.1677], 
                 tiles='cartodbpositron', 
                 zoom_start=8)

# Add bike stations to the map
for idx, row in grid_plot.iterrows():
    folium.Circle(location=[row.lat, row.lon],
                  radius=20,
                  color='black').add_to(map1)

# Add weather stations
folium.Marker([43.6532, -79.3832], icon=folium.Icon(color='blue'), popup='UofT').add_to(map1)
folium.Marker([43.9593, -78.1677], icon=folium.Icon(color='red'), popup='Facility').add_to(map1)

# Display map
map1

## Question 1
### HTML & Web Scraping
For Question 1, you'll be using your web scraping skillz to create a reference DataFrame with the names of files that contain wave data for Lake Ontario in 2021.The files can be found [here](https://www.glerl.noaa.gov/emf/waves/GLERL-Donelan-Archive/2021/').

The filename format is:
LYYYY_MM.F.NC

where:
- L = lake letter (c=St. Clair, s=Superior, m=Michigan, h=Huron, e=Erie, o=Ontario)
- YYYY = year at start of simulation (GMT)
- MM = month at start of simulation (GMT)
- F = is either `in1` or `out1` (don't worry about this)  
- NC = file extension

### Question 1a
Use `requests.get()` to grab the HTML from this link (https://www.glerl.noaa.gov/emf/waves/GLERL-Donelan-Archive/2021/). Create a new variable named `response` and assign the returned object from `requests.get()` to it. Sometimes the **GLERL** server doesn't return a vaid response, which produces a Python error. You may have to run this cell more than once.

In [70]:
# assess the NOAA document archive using the requests library function
response = requests.get('https://www.glerl.noaa.gov/emf/waves/GLERL-Donelan-Archive/2021/')

# use the status_code (200 = success) to confirm the NOAA retrieved data
response.status_code

200

Using `BeautifulSoup` to parse the html object returned by `requests.get()`.

In [71]:
soup = BeautifulSoup(response.text, 'html.parser')

Using the `.findAll()` method to generate a list of HTML entries for each NOAA file.

In [72]:
table_rows = soup.findAll('tr')

The table data `<td>` we're interested in only appears after line 3 so let's grab everything from the 3th row on. 

In [73]:
table_rows = table_rows[3:-1]

Print the first 5 rows of `table_rows` to see what they contain.

In [74]:
for table_row in table_rows[0:5]:
    print(table_row)

<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="c2021.nc">c2021.nc</a></td><td align="right">2022-02-15 14:20  </td><td align="right">779M</td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="c2021_05.in1.nc">c2021_05.in1.nc</a></td><td align="right">2021-10-13 16:00  </td><td align="right">111M</td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="c2021_06.in1.nc">c2021_06.in1.nc</a></td><td align="right">2021-10-13 16:01  </td><td align="right">108M</td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="c2021_07.in1.nc">c2021_07.in1.nc</a></td><td align="right">2021-10-13 16:04  </td><td align="right">111M</td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="c2021_08.in1.nc">c2021_08.in1.nc</a></td><td align="right">2021-10-13 16:05  </td><td align="right">111M

In [75]:
# to find the total number of rows in the table_rows dataset:
# use the len function
len(table_rows)

54

The `table_rows` contains `<tr></tr>` and `<td></td>` html tags. Each list entry is wrapped in `<tr></tr>` tags and within these `<tr></tr>` tags, the data needed in is wrapped in `<td></td>` tags.

### Question 1b
Use `table_rows` to create a DataFrame that inludes a row entry for each list item in `table_rows`. There are 54 list items in `table_rows` and therefore, you'll be creating a DataFrame with 54 rows. Create a variable called `noaa_files` and assign the DataFrame to it. The DataFrame should have three columns `filename`, `upload_date`, and `file_size`, which can all be extracted from the HTML snippets in `table_rows`. Make sure to use string method to remove any excess white space. For example, there may be a filename string `'	  o2021_12.in1.nc'` but it should be entered into the DataFrame as `'co2021_12.in1.nc'`. 

`noaa_file.head(10)` should return something like this but with different `filename`, `upload_date`, and `file_size`. The table below is just to give you an idea for is expected but yours will look different.
<br>
<img src="images/noaa_files.png" alt="drawing" width="350"/>
<br>

In [95]:
# create three different columns- the filename, upload_date, file_size
# create three empty lists where each of the list is for each column

filename=[]
upload_date=[]
file_size=[]

# each of the rows in the dataset table_rows is parsed
# each row will provide information for the creation of the three columns
# the information for each of the column is delimited between each 'td' tag
# use the findAll function to identify the tags in each row.
# use the append function to append the tagged part of the row to the list representing the column

for i in range(0,len(table_rows)):
    filename.append(table_rows[i].findAll('a')[0].contents)
    upload_date.append(table_rows[i].findAll('td')[2].contents)
    file_size.append(table_rows[i].findAll('td')[3].contents)

# after splitting all the rows in table_rows in the list columns, this creates a new dataframe which uses the columns as data entries

noaa_files = pd.DataFrame(
    {'filename':filename,
     'upload_date': upload_date,
     'file_size': file_size
    })
                        
# inorder to clean the files, change all the dataframe entries from object to string type using the astype function
noaa_files=noaa_files.astype(str)

# use the string strip function to remove excess white space or extra information
noaa_files['filename']=noaa_files['filename'].str.strip("[]'")
noaa_files['upload_date']=noaa_files['upload_date'].str.split("").str.get(1)
noaa_files['file_size']=noaa_files['file_size'].str.split("").str.get(1)

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row[1:4])
    
# check the output of the dataframe by printing the column names and first ten entries using the head function
noaa_files = pd.DataFrame(l, columns=["filename", "upload_date", "file_size"])
noaa_files.head(10)


,filename,upload_date,file_size
0,c2021.nc,2022-02-15 14:20,779M
1,c2021_05.in1.nc,2021-10-13 16:00,111M
2,c2021_06.in1.nc,2021-10-13 16:01,108M
3,c2021_07.in1.nc,2021-10-13 16:04,111M
4,c2021_08.in1.nc,2021-10-13 16:05,111M
5,c2021_09.in1.nc,2021-10-13 16:05,107M
6,c2021_10.in1.nc,2021-11-01 05:44,111M
7,c2021_11.in1.nc,2021-11-30 13:26,107M
8,c2021_12.in1.nc,2021-12-31 13:26,110M
9,e2021.nc,2022-02-15 14:20,1.6G


Remember, `noaa_files` should have 54 rows. Let's check.

In [37]:
print('noaa_files has {} rows'.format(noaa_files.shape[0]))

noaa_files has 54 rows


### Question 1c
Filter `noaa_files` so it only contains `.in1.nc` files for lake Ontario. After filtering, reset the index using `.reset_index(drop=True)`.

In [99]:
# use the loc method to filter the dataframe to rows
# use string contains funtion to check for file names that starts with o and contains out1.nc in the dataframe,
# then filter for Lake Ontario files using .out1.nc function

noaa_files=noaa_files.loc[noaa_files['filename'].str.contains("o" and '.in1.nc')].reset_index(drop=True)

# Use the head function to view the dataframe
noaa_files.head(8)

,filename,upload_date,file_size
0,o2021_05.in1.nc,2021-10-13 16:00,22M
1,o2021_06.in1.nc,2021-10-13 16:01,21M
2,o2021_07.in1.nc,2021-10-13 16:03,22M
3,o2021_08.in1.nc,2021-10-13 16:04,22M
4,o2021_09.in1.nc,2021-10-13 16:05,21M
5,o2021_10.in1.nc,2021-11-01 05:43,22M
6,o2021_11.in1.nc,2021-11-30 13:26,21M
7,o2021_12.in1.nc,2021-12-31 13:26,21M


`noaa_files` should now have 8 rows. Let's check.

In [16]:
print('noaa_files has {} rows'.format(noaa_files.shape[0]))

noaa_files has 54 rows


## Question 2
Now that we have a DataFrame containing the names of all the files we're interested in, we can start downloading these files and extracting the desired information. Because the **GLERL** server can be a bit unreliable and because they recently retired this database, we've gone ahead and downloaded a bunch of `.wav` for you, which are located in the assignment directory. 

For Question 2, we are working with a different file because of the recent database changes. For Question 2 we are working with `.wav` (Wave) files with the following naming conversion.

The gridded fields filename format is:
LYYYYDDDHH.N.WAV

where:
- L = lake letter (s=Superior, m=Michigan, h=Huron, e=Erie, o=Ontario)
- YYYY = year at start of simulation (GMT)
- DDD = Day Of Year at start of simulation (GMT)
- HH = hr at start of simulation (GMT)
- N = Site Number (don't worry about this, it will always be 0)

Because we are focused on wave information for Lake Ontario, our files have the following format **oYYYYDDDHH.0.wav** (`o` for Lake Ontario and `.wav` for wave measurements).

Let's import the `noaa_files` `.csv` that corresponds to the files we've already downloaded for you.

In [17]:
noaa_files = pd.read_csv('noaa_files.csv')
noaa_files.head()

,filename,upload_date,file_size
0,o202101518.0.wav,2021-01-15 19:23,110K
1,o202101600.0.wav,2021-01-16 01:23,110K
2,o202101606.0.wav,2021-01-16 07:23,110K
3,o202101612.0.wav,2021-01-16 13:23,110K
4,o202101618.0.wav,2021-01-16 19:23,110K


Let's also take a look at the `.wav` files we've included for you in the assignment folder.

In [18]:
[file for file in os.listdir() if '.wav' in file]

['o202102312.0.wav',
 'o202102306.0.wav',
 'o202102300.0.wav',
 'o202102218.0.wav',
 'o202102212.0.wav',
 'o202102206.0.wav',
 'o202102200.0.wav',
 'o202102118.0.wav',
 'o202102112.0.wav',
 'o202102106.0.wav',
 'o202102100.0.wav',
 'o202102018.0.wav',
 'o202102012.0.wav',
 'o202102006.0.wav',
 'o202102000.0.wav',
 'o202101918.0.wav',
 'o202101912.0.wav',
 'o202101906.0.wav',
 'o202101900.0.wav',
 'o202101818.0.wav',
 'o202101812.0.wav',
 'o202101806.0.wav',
 'o202101800.0.wav',
 'o202101718.0.wav',
 'o202101712.0.wav',
 'o202101706.0.wav',
 'o202101700.0.wav',
 'o202101618.0.wav',
 'o202101612.0.wav',
 'o202101606.0.wav',
 'o202101600.0.wav',
 'o202101518.0.wav']

There are 32 rows in the `noaa_files` DataFrame and 32 `.wav` files in the assignment folder. They correspond to the same 32 datetimes.

### Question 2a
Write a function `noaa_file_parser(filename)` that takes a filename as as an argument (e.g. filename = `o202101518.0.wav`) and returns a DataFrame with the following columns.

- filename (string)
- year (int)
- day (int)
- hour (int)
- grid_number (int)
- wave_height (float)
- wave_direction (int)
- wave_period (float)

There are two types of rows in a noaa file (e.g. `o202101518.0.wav`):

1. **Time stamp row** - this row indicates the date and time of the grid measurements to follow.
    
    Example: For file `o202101518.0.wav`, the first row is 
    
    `'2021 015 19     /glcfs/bathy/ontario5km.dat    WAVES                   746'`
    
    There are six entries in a time stamp row.
    
    - Year (GMT)
    - Day of the year (GMT)
    - Hour of the day (GMT)
    - Map file. There is a map file for each lake and it relates the grid numbers to latitudes and longitudes. `ontario5km.map` is the Lake Ontario map and its already in your assignment folder.
    - File type (WAVES)
    - Number of grid points for a particular lake (746 for lake ontario).

2. **Measurement row** - this row contains the wave measurements we're interested in. Each measurement row contains wave measurements for a particular gird point. 

    Example: For file `o202101518.0.wav`, the second row is 
    
    `'1   1.029  231  4.2'`
    
    There are four entries in a measurement row.
    
    - grid_number (int)
    - wave height (meters) (float)
    - wave direction (0 = toward north, 90 = toward east) (int)
    - wave period (s) (float)
    
The NOAA files contain this repeating pattern:
```
2021 015 19     /glcfs/bathy/ontario5km.dat    WAVES                   746
1   1.029  231  4.2
2   0.932  228  4.0
...
745   0.656  312  3.4
746   0.581  331  3.4
2021 015 20     /glcfs/bathy/ontario5km.dat    WAVES                   746
1   1.142  228  4.4
2   1.042  227  4.3
...
745   0.642  316  3.4
746   0.600  345  3.8
2021 015 21     /glcfs/bathy/ontario5km.dat    WAVES                   746
1   1.205  224  4.9
2   1.109  223  4.7
...
745   0.635  317  3.4
746   0.592  350  3.8
```
    
Your function `noaa_file_parser(filename='o202101518.0.wav')` should return the following DataFrame (first 10 rows shown). 
<br>
<img src="images/noaa_file_parser.png" alt="drawing" width="550"/>
<br>

You can use the code in the cell below to explore NOAA file `'o202101518.0.wav'`.

In [19]:
row=open('o202101518.0.wav', 'r').read().split('\n')[0:10]
row

['2021 015 19     /glcfs/bathy/ontario5km.dat    WAVES                   746',
 '     1   1.029  231  4.2',
 '     2   0.932  228  4.0',
 '     3   0.816  225  3.8',
 '     4   0.648  221  3.5',
 '     5   1.347  237  4.7',
 '     6   1.282  236  4.6',
 '     7   1.207  235  4.5',
 '     8   1.121  234  4.3',
 '     9   1.039  233  4.2']

In [20]:
# noaa_file_parser function is defined, that takes a filename as input and outputs dataframe as the result
def noaa_file_parser(filename):
    
    #the file name is opened and stored as a single long string, where each line is split on the new-line character
    file=open(filename, 'r').read().split('\n')
    
    #two lists are defined in the code, which will be split later to define the columns of the result dataframe:
    #1 - data_year_hour_list = list comprehension is used to extract every 747th header line in the file to this list
    #2 - wave_grid_list = list comprehension is used to dump all the lines in the file to this list. Header files will be 
    #    filtered out later when moving the list to a dataframe
    date_year_hour_list = [x for x in file[0::747]]
    wave_grid_list = [x for x in file[0:]]
    
    #the date_year_hour_list is put into a panda's dataframe. Initially all the list entries are put into one column called
    # 'data'. The header entries in data are delimited by a white-space, which is used to define the three columns "year", "day"
    # and "hour". Extra leading characters, such as the zero in "day" is removed using the left-strip method.
    
    date_year_hour_df = pd.DataFrame(date_year_hour_list,columns=['data'])
    date_year_hour_df = date_year_hour_df['data'].str.split(' ',expand=True)
    date_year_hour_df = date_year_hour_df[[0,1,2]].rename(columns={0: "year", 1: "day", 2: "hour"})
    date_year_hour_df['day'] = date_year_hour_df['day'].str.lstrip('0')
    
    #the size of the header data-frame is not the same size as wave_grid information. The index repeat method is used 
    #to sequentially repeat the correct information and resize the dataframe, reset_index updates to a new index of the 
    #correct data-frame size.
    date_year_hour_df=date_year_hour_df.loc[date_year_hour_df.index.repeat(746)].reset_index()
     
    #a second dataframe is created, wave_grid_list_df, using the other list, wave_grid_list, the same as above. To remove
    #the header rows inside this dataframe, the modulus function is used to identify every 747th row and then drop these values.
    #the index is reset to update the dataframe.
    wave_grid_list_df = pd.DataFrame(wave_grid_list,columns=['data'])
    wave_grid_list_df = wave_grid_list_df[wave_grid_list_df.index % 747 != 0].reset_index(drop=True)
    
    #the wave information is also white-space delimited, and the same string split method is used to expand the data into 
    #multiple columns, grid_number and wave, on a two-space white-space delimiter. The wave information can be subdivided
    #into three more columns, based on a single space delimiter, wave_height, wave_direction, wave_period.
    wave_grid_list_df=wave_grid_list_df['data'].str.split("   ",expand=True)
    wave_grid_list_df = wave_grid_list_df[[1,2]].rename(columns={1: "grid_number", 2: "wave"})
    wave_grid_list_df[["wave_height", "wave_direction", "wave_period"]] = wave_grid_list_df['wave'].str.split("  ", expand=True)
    
    #once the two data-frames have the correct columns and are the correct size with matching indices, a join can be performed on 
    #the two dataframes to combine them into a new dataframe file_grid_data. After the data-frames are combined, a new column
    #filename is created by noting the original function argument. 
    #The dataframe order and required columns are specified, and finally the to_numeric funnction is used to transform the string
    #values in the dataframe into float/int values, so they can be used for future calculations.
    #This dataframe is returned as the function result.
    file_grid_data=date_year_hour_df.join(wave_grid_list_df, lsuffix='_date-year', rsuffix='_wave-grid')
    file_grid_data['filename']=filename
    file_grid_data = file_grid_data[['filename','grid_number','year','day','hour','wave_height','wave_direction','wave_period']]
    file_grid_data[["grid_number","wave_height","wave_direction","wave_period"]] = file_grid_data[["grid_number","wave_height","wave_direction","wave_period"]].apply(pd.to_numeric)
    
    return file_grid_data

# Print DataFrame head. Here I am using the first file in the assignment directory to print (o202101518.0.wav).
file_grid_data = noaa_file_parser(filename=noaa_files.loc[0, 'filename'])
file_grid_data.head(10)

,filename,grid_number,year,day,hour,wave_height,wave_direction,wave_period
0,o202101518.0.wav,1,2021,15,19,1.029,231.0,4.2
1,o202101518.0.wav,2,2021,15,19,0.932,228.0,4.0
2,o202101518.0.wav,3,2021,15,19,0.816,225.0,3.8
3,o202101518.0.wav,4,2021,15,19,0.648,221.0,3.5
4,o202101518.0.wav,5,2021,15,19,1.347,237.0,4.7
5,o202101518.0.wav,6,2021,15,19,1.282,236.0,4.6
6,o202101518.0.wav,7,2021,15,19,1.207,235.0,4.5
7,o202101518.0.wav,8,2021,15,19,1.121,234.0,4.3
8,o202101518.0.wav,9,2021,15,19,1.039,233.0,4.2
9,o202101518.0.wav,10,2021,15,19,0.955,231.0,4.1


`file_grid_data` should have 4476 rows. Let's check

In [21]:
# use the 0th index of shape function to determine the number of rows in the result of the dataframe
print('file_grid_data has {} rows'.format(file_grid_data.shape[0]))

file_grid_data has 4476 rows


### Question 2b
Next, use the function you just built (`noaa_file_parser()`) to parse each noaa file in the DataFrame `noaa_files`. Run `noaa_file_parser()` in a `for` loop that loops through each file in `noaa_files['filename]`. At the end of each loop, add the DataFrame returned by `noaa_file_parser()` to a list. After looping through every file in the DataFrame `noaa_files`, use `pd.concat()` to combine all the DataFrames. The result will be one large DataFrame containing all the wave measurement data. Create a variable `grid_data` and assign this new DataFrame to it.

In [23]:
# create an empty list to initially save the dataframes returned by the noaa_file_parser
storage=[]

# all the files in noaa_files are iterated by the loop function through the use of the range function running from 0 to the length of the noaa_files. 
# use i as the function for each iterated file in the noaa_files
# the dataframe results are appended to the storage list.
for i in range(0,len(noaa_files)):
    storage.append(noaa_file_parser(filename=noaa_files.loc[i, 'filename']))
               
# use the concat function to combine all the dataframes from the saved storage into a larger dataframe (single)
# view dataframe by viewing the first five entries using the head method
grid_data = pd.concat(storage)
grid_data.head() 

,filename,grid_number,year,day,hour,wave_height,wave_direction,wave_period
0,o202101518.0.wav,1,2021,15,19,1.029,231.0,4.2
1,o202101518.0.wav,2,2021,15,19,0.932,228.0,4.0
2,o202101518.0.wav,3,2021,15,19,0.816,225.0,3.8
3,o202101518.0.wav,4,2021,15,19,0.648,221.0,3.5
4,o202101518.0.wav,5,2021,15,19,1.347,237.0,4.7


As a quick check, `grid_data` should have 32 unique files and 143232 rows in it.

In [24]:
print('There are {} unique files in grid_data and {} rows'.format(grid_data['filename'].nunique(), grid_data.shape[0]))

There are 32 unique files in grid_data and 143232 rows


### Question 2c
The map file `ontario5km.map` is located in the assignment directory and contains the latitude and longitude information for each grid_number in `grid_data`. Import the `ontario5km.map` as a DataFrame with the following column names:

- grid_number
- fortran column
- fortran row
- lat
- lon
- depth

The ascii map file structure is:

NNNNN III JJJ LL.LLLLL LL.LLLLL DDD

where:
- NNNNN = sequence number (grid_number)
- III = fortran column
- JJJ = fortran row
- LL.LLLLL = lat (decimal degrees N)
- LL.LLLLL = lon (decimal degrees W)
- DDD = depth (m)

**Hint:** The columns are white-space delimited.

Create a variable `map_file` and assign this new DataFrame to it.

Below is a quick view of what the file `'ontario5km.map'` contains. Notice there are no column names so we'll have to add them ourselves.

In [25]:
open('ontario5km.map', 'r').read().split('\n')[0:10]

['   1 006 002 43.19300 79.48237   8.0',
 '   2 007 002 43.19380 79.42086   8.0',
 '   3 008 002 43.19456 79.35935   9.0',
 '   4 009 002 43.19529 79.29785   8.0',
 '   5 003 003 43.23541 79.66814   8.0',
 '   6 004 003 43.23631 79.60659  11.0',
 '   7 005 003 43.23717 79.54504  18.0',
 '   8 006 003 43.23800 79.48348  26.0',
 '   9 007 003 43.23879 79.42194  33.0',
 '  10 008 003 43.23955 79.36038  33.0']

In [26]:
#The variable file is used to store the results of the map file in a single string, which is split based on the new-line
#character. List comprehension is used to store this data in a single list, called ontario_map, with index slices based on each row.
file = open('ontario5km.map', 'r').read().split('\n')
ontario_map = [x for x in file[0:]]
    
#A new data frame is defined based on the ontaro_map list as the data, called map_file. The data is put into a single column 
#called data. Leading white spaces are removed using the left-strip (lstrip) method. The data in each row is delimited by a
#white-space, which is split using the string split method on white space and expanding the information in data across
#multiple columns. These columns are identified as: grid_number, fortran column, fortran row, lat, lon and depth.
map_file = pd.DataFrame(ontario_map,columns=['data'])
map_file['data'] = map_file['data'].str.lstrip()
map_file = map_file['data'].str.split(' ',expand=True)
map_file = map_file[[0,1,2,3,4,7]].rename(columns={0: "grid_number", 1: "fortran column", 2: "fortran row", 3: "lat", 4: "lon", 7:"depth"})

#The grid_number, lat and lon inside the dataframe are converted from string to float/int by using the pandas to_numeric methods, so 
#future calculations can be performed on these columns.
map_file[["grid_number","lat", "lon"]] = map_file[["grid_number","lat", "lon"]].apply(pd.to_numeric)

#Because Longitude is in units of (decimal degrees W), these are converted to negative values 
#by multiplying the column by -1.
map_file['lon'] = map_file['lon'] * -1

# View DataFrame by displaying the first five entries using the head function
map_file.head()

,grid_number,fortran column,fortran row,lat,lon,depth
0,1,006,002,43.19300,-79.48237,8.0
1,2,007,002,43.19380,-79.42086,8.0
2,3,008,002,43.19456,-79.35935,9.0
3,4,009,002,43.19529,-79.29785,8.0
4,5,003,003,43.23541,-79.66814,8.0


### Question 2d
We want our wave measurements in `grid_data` to have a geographic location so we can find the 3 closest grid points to the planned facility. To do this, we must use `pd.merge()` to map columns `lat` and `lon` from `map_file` to `grid_data`. Create a variable `grid_data_final` and assign this new DataFrame to it.

`grid_data_final.head()` should look like this:
<br>
<img src="images/grid_data.png" alt="drawing" width="700"/>
<br>

In [27]:
# to make the dataframe grid_data_final, use the pandas merge function to join the grid_data and the map_file dataframe
# to identify the unique key columns between the two dataframes (grid_number), use the on function in the merge method. 
# resave the dataframe with specific columns to reduce the dataframe to just the columns of interest
grid_data_final = grid_data.merge(map_file,on='grid_number')
grid_data_final = grid_data_final[['filename','year','day','hour','grid_number','wave_height','wave_direction','wave_period','lat','lon']]

# View DataFrame
grid_data_final.head()

,filename,year,day,hour,grid_number,wave_height,wave_direction,wave_period,lat,lon
0,o202101518.0.wav,2021,15,19,1,1.029,231.0,4.2,43.193,-79.48237
1,o202101518.0.wav,2021,15,20,1,1.142,228.0,4.4,43.193,-79.48237
2,o202101518.0.wav,2021,15,21,1,1.205,224.0,4.9,43.193,-79.48237
3,o202101518.0.wav,2021,15,22,1,1.159,221.0,5.3,43.193,-79.48237
4,o202101518.0.wav,2021,15,23,1,0.893,225.0,4.6,43.193,-79.48237


### Question 2e
Save `grid_data_final` to the root path with file name `'grid_data_final.csv'`. Make sure to not include an index column.

In [29]:
# save grid_data_final to a local csv file using the to_csv built in function, removing the index
grid_data_final.to_csv('grid_data_final.csv', index=False)

### Question 2f
Next, we need to find the three grid points that are closest to the location of the planned facility. The facility will be located at (lat = 43.9593°, lon = -78.1677°). A helper function has been included to calculate the distance between any two points (lat1, lon1 and lat2, lon2). We’ll use the Haversine (or Great Circle) distance formula, which takes the latitude and longitude of two points, adjusts for Earth’s curvature, and calculates the straight-line distance between them. 

You can call this function as follows:

`utils.haversine(lat1, lon1, lat2, lon2)`

The distance is returned in kilometers.

First, create a new column called `distance` for DataFrame `grid_data_final`. You can use the `.apply()` method to apply `utils.haversine(lat1, lon1, lat2, lon2)` to each row. 

`grid_data_final.head()` should look like this:
<br>
<img src="images/grid_data_distance.png" alt="drawing" width="700"/>
<br>

In [32]:
# save the facility latitude and longitude to variables facility_lat and facility_lon respectively
# a new column distance is added to the grid_data_final dataframe
# use lambda and apply to make sure that the haversine function is applied along each row and to maintain the correct shape of the dataframe.            
facility_lat = 43.9593
facility_lon = -78.1677
grid_data_final['distance'] = grid_data_final.apply(lambda row: utils.haversine(row['lat'], row['lon'], facility_lat, facility_lon), axis=1)

# View DataFrame
grid_data_final.head()

,filename,year,day,hour,grid_number,wave_height,wave_direction,wave_period,lat,lon,distance
0,o202101518.0.wav,2021,15,19,1,1.029,231.0,4.2,43.193,-79.48237,135.932581
1,o202101518.0.wav,2021,15,20,1,1.142,228.0,4.4,43.193,-79.48237,135.932581
2,o202101518.0.wav,2021,15,21,1,1.205,224.0,4.9,43.193,-79.48237,135.932581
3,o202101518.0.wav,2021,15,22,1,1.159,221.0,5.3,43.193,-79.48237,135.932581
4,o202101518.0.wav,2021,15,23,1,0.893,225.0,4.6,43.193,-79.48237,135.932581


### Question 2g
Next, create a new DataFrame called `closest_points` which only contains wave measurement data from the three closest grid points to the planned facility at (lat = 43.9593°, lon = -78.1677°).

In [33]:
#A new dataframe, closest_points, is defined by making a copy of the dataframe grid_data_final. All values in grid_data_final
#that have an empty distance value are dropped (as we are looking only for the shortest distances, and want to avoid null
#values that could skew the data) and the index is reset. The values in closest_points are sorted by their distance 
#value, in ascending order from smallest to largest.
#A filter unique_id is defined, which finds the first three smallest distances in the closest_points dataframe, and this 
#is used to trim the dataframe to include only the three closest grid points to the facility.
closest_points = (grid_data_final.dropna(subset='distance')).sort_values(by='distance',ascending=True).reset_index(drop=True)
unique_id = closest_points["distance"].unique()[0:3]
closest_points=closest_points.loc[closest_points['distance'].isin(unique_id)]

# View DataFrame by displaying the first five entries by using the head function 
closest_points.head() 

,filename,year,day,hour,grid_number,wave_height,wave_direction,wave_period,lat,lon,distance
0,o202102312.0.wav,2021,23,18,659,0.429,147.0,2.5,43.92241,-78.19302,4.575873
1,o202101812.0.wav,2021,18,15,659,0.240,176.0,1.9,43.92241,-78.19302,4.575873
2,o202101812.0.wav,2021,18,14,659,0.199,170.0,1.8,43.92241,-78.19302,4.575873
3,o202101812.0.wav,2021,18,13,659,0.207,163.0,1.8,43.92241,-78.19302,4.575873
4,o202101806.0.wav,2021,18,12,659,0.244,160.0,2.0,43.92241,-78.19302,4.575873


As a quick check, `closest_points` should have 576 rows and 3 unique `'grid_number'`. 

In [34]:
print('closest_points has {} rows and {} unique grid_number.'.format(closest_points.shape[0], closest_points['grid_number'].nunique()))

closest_points has 576 rows and 3 unique grid_number.


Validate that the three points you've found make sense visually with a plot. The three closest point are shown as red markers.

In [40]:
# Import 'grid_plot.csv'
grid_plot = pd.read_csv('grid_plot.csv')

# Create a map of Toronto
map2 = folium.Map(location=[43.9593, -78.1677], 
                 tiles='cartodbpositron', 
                 zoom_start=8)

# Add bike stations to the map
for idx, row in grid_plot.iterrows():
    folium.Circle(location=[row.lat, row.lon],
                  radius=20,
                  color='black').add_to(map2)

# Add bike stations to the map
for idx, row in closest_points.iterrows():
    folium.Circle(location=[row.lat, row.lon],
                  radius=20,
                  color='red').add_to(map2)
    
# Add weather stations
folium.Marker([43.6532, -79.3832], icon=folium.Icon(color='blue'), popup='UofT').add_to(map2)
folium.Marker([43.9593, -78.1677], icon=folium.Icon(color='red'), popup='Facility').add_to(map2)

# Display map
map2

# from the data displayed below: there are three red dots highlighted close to the red locator which represents the facility. 
# this shows that the results calculated in the dataframe closest_points were correct and accurate.

### Question 2h
The final step is to use the `closest_points` DataFrame to calculate the minimum, maximum and average wave height across the three closest points. Create variables `wave_height_min`, `wave_height_max`, `wave_height_mean` and assign the computed values to them.

In [41]:
# to find the minimum, maximum and average values in the wave_height:

# apply the min() function directly to the wave_height column in the closest_points dataframe.
wave_height_min = closest_points['wave_height'].min()

# apply the max() function directly to the wave_height column in the closest_points dataframe.
wave_height_max = closest_points['wave_height'].max()

# apply the mean() function directly to the wave_height column in the closest_points dataframe.
wave_height_mean = closest_points['wave_height'].mean()


# Print answers
print('Wave height min: {} m\nWave height max: {} m\nWave height mean: {} m'.format(wave_height_min, 
                                                                                    wave_height_max, 
                                                                                    wave_height_mean)) 

Wave height min: 0.063 m
Wave height max: 2.629 m
Wave height mean: 0.9317013888888889 m
